In [6]:
!pip install lazypredict

In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import ExtraTreesClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [9]:
from lazypredict.Supervised import LazyClassifier
import pickle
import joblib

TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'

In [ ]:
train_data=pd.read_csv("/content/supply_chain_data.csv")
train_data

,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,...,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs
0,haircare,SKU0,69.81,55,802,8662.00,Non-binary,58,7,96,...,Mumbai,29,215,29,46.28,Pending,0.23,Road,Route B,187.75
1,skincare,SKU1,14.84,95,736,7460.90,Female,53,30,37,...,Mumbai,23,517,30,33.62,Pending,4.85,Road,Route B,503.07
2,haircare,SKU2,11.32,34,8,9577.75,Unknown,1,10,88,...,Mumbai,12,971,27,30.69,Pending,4.58,Air,Route C,141.92
3,skincare,SKU3,61.16,68,83,7766.84,Non-binary,23,13,59,...,Kolkata,24,937,18,35.62,Fail,4.75,Rail,Route A,254.78
4,skincare,SKU4,4.81,26,871,2686.51,Non-binary,5,3,56,...,Delhi,5,414,3,92.07,Fail,3.15,Air,Route A,923.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,haircare,SKU95,77.90,65,672,7386.36,Unknown,15,14,26,...,Mumbai,18,450,26,58.89,Pending,1.21,Air,Route A,778.86
96,cosmetics,SKU96,24.42,29,324,7698.42,Non-binary,67,2,32,...,Mumbai,28,648,28,17.80,Pending,3.87,Road,Route A,188.74
97,haircare,SKU97,3.53,56,62,4370.92,Male,46,19,4,...,Mumbai,10,535,13,65.77,Fail,3.38,Road,Route A,540.13
98,skincare,SKU98,19.75,43,913,8525.95,Female,53,1,27,...,Chennai,28,581,9,5.60,Pending,2.91,Rail,Route A,882.20


In [ ]:
train_data.isnull().sum()

Product type               0
SKU                        0
Price                      0
Availability               0
Number of products sold    0
Revenue generated          0
Customer demographics      0
Stock levels               0
Lead times                 0
Order quantities           0
Shipping times             0
Shipping carriers          0
Shipping costs             0
Supplier name              0
Location                   0
Lead time                  0
Production volumes         0
Manufacturing lead time    0
Manufacturing costs        0
Inspection results         0
Defect rates               0
Transportation modes       0
Routes                     0
Costs                      0
dtype: int64

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Product type             100 non-null    object 
 1   SKU                      100 non-null    object 
 2   Price                    100 non-null    float64
 3   Availability             100 non-null    int64  
 4   Number of products sold  100 non-null    int64  
 5   Revenue generated        100 non-null    float64
 6   Customer demographics    100 non-null    object 
 7   Stock levels             100 non-null    int64  
 8   Lead times               100 non-null    int64  
 9   Order quantities         100 non-null    int64  
 10  Shipping times           100 non-null    int64  
 11  Shipping carriers        100 non-null    object 
 12  Shipping costs           100 non-null    float64
 13  Supplier name            100 non-null    object 
 14  Location                 10

In [ ]:
labelencoder=LabelEncoder()
def object_to_int(data):
    if data.dtypes=='object':
        data=labelencoder.fit_transform(data)
    return data

In [ ]:
train_data=train_data.apply(lambda data:object_to_int(data))
train_data

,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,...,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs
0,1,0,69.81,55,802,8662.00,2,58,7,96,...,4,29,215,29,46.28,2,0.23,2,1,187.75
1,2,1,14.84,95,736,7460.90,0,53,30,37,...,4,23,517,30,33.62,2,4.85,2,1,503.07
2,1,12,11.32,34,8,9577.75,3,1,10,88,...,4,12,971,27,30.69,2,4.58,0,2,141.92
3,2,23,61.16,68,83,7766.84,2,23,13,59,...,3,24,937,18,35.62,0,4.75,1,0,254.78
4,2,34,4.81,26,871,2686.51,2,5,3,56,...,2,5,414,3,92.07,0,3.15,0,0,923.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,95,77.90,65,672,7386.36,3,15,14,26,...,4,18,450,26,58.89,2,1.21,0,0,778.86
96,0,96,24.42,29,324,7698.42,2,67,2,32,...,4,28,648,28,17.80,2,3.87,2,0,188.74
97,1,97,3.53,56,62,4370.92,1,46,19,4,...,4,10,535,13,65.77,0,3.38,2,0,540.13
98,2,98,19.75,43,913,8525.95,0,53,1,27,...,1,28,581,9,5.60,2,2.91,1,0,882.20


In [ ]:
train_data['Customer_demand'] = [1 if (int(column['Stock levels'] - column['Order quantities']) < 0) else 0 for _,column in train_data.iterrows()]
train_data

,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,...,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs,Customer_demand
0,1,0,69.81,55,802,8662.00,2,58,7,96,...,29,215,29,46.28,2,0.23,2,1,187.75,1
1,2,1,14.84,95,736,7460.90,0,53,30,37,...,23,517,30,33.62,2,4.85,2,1,503.07,0
2,1,12,11.32,34,8,9577.75,3,1,10,88,...,12,971,27,30.69,2,4.58,0,2,141.92,1
3,2,23,61.16,68,83,7766.84,2,23,13,59,...,24,937,18,35.62,0,4.75,1,0,254.78,1
4,2,34,4.81,26,871,2686.51,2,5,3,56,...,5,414,3,92.07,0,3.15,0,0,923.44,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,95,77.90,65,672,7386.36,3,15,14,26,...,18,450,26,58.89,2,1.21,0,0,778.86,1
96,0,96,24.42,29,324,7698.42,2,67,2,32,...,28,648,28,17.80,2,3.87,2,0,188.74,0
97,1,97,3.53,56,62,4370.92,1,46,19,4,...,10,535,13,65.77,0,3.38,2,0,540.13,0
98,2,98,19.75,43,913,8525.95,0,53,1,27,...,28,581,9,5.60,2,2.91,1,0,882.20,0


In [ ]:
X=train_data.drop(['Customer_demand'],axis=1)
y=train_data['Customer_demand']

In [ ]:
X = X.head(70)
y = y.head(70)

In [ ]:
f_p_value=chi2(X,y)

In [ ]:
p_value=pd.Series(f_p_value[1])
p_value.index=X.columns
p_value.sort_index(ascending=True)

Availability              0.00
Costs                     0.00
Customer demographics     0.68
Defect rates              0.97
Inspection results        0.40
Lead time                 0.07
Lead times                0.64
Location                  0.52
Manufacturing costs       0.12
Manufacturing lead time   0.29
Number of products sold   0.00
Order quantities          0.00
Price                     0.01
Product type              0.52
Production volumes        0.00
Revenue generated         0.00
Routes                    0.80
SKU                       0.71
Shipping carriers         0.33
Shipping costs            0.19
Shipping times            0.26
Stock levels              0.00
Supplier name             0.08
Transportation modes      0.69
dtype: float64

In [ ]:
X.drop(['Routes','Shipping carriers','Shipping costs','Shipping times','Supplier name','Transportation modes'],axis=1,inplace=True)

In [ ]:
X.drop(['Location','Revenue generated','Manufacturing costs'],axis=1,inplace=True)

In [ ]:
test_data = X.tail(30)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [ ]:
clf = LazyClassifier()
model, predict = clf.fit(X_train, X_test, y_train, y_test)
model

100%|██████████| 29/29 [00:01<00:00, 21.27it/s]

[LightGBM] [Info] Number of positive: 26, number of negative: 23
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000051 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 212
[LightGBM] [Info] Number of data points in the train set: 49, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.530612 -> initscore=0.122602
[LightGBM] [Info] Start training from score 0.122602
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
Perceptron,0.95,0.96,0.96,0.95,0.01
LogisticRegression,0.95,0.96,0.96,0.95,0.02
AdaBoostClassifier,0.90,0.92,0.92,0.91,0.11
BaggingClassifier,0.90,0.92,0.92,0.91,0.04
XGBClassifier,0.90,0.92,0.92,0.91,0.45
SGDClassifier,0.90,0.92,0.92,0.91,0.01
LinearSVC,0.90,0.92,0.92,0.91,0.01
CalibratedClassifierCV,0.90,0.92,0.92,0.91,0.03
DecisionTreeClassifier,0.90,0.92,0.92,0.91,0.01


In [ ]:
xgb = XGBClassifier()
xgb.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
print("Training Accuracy: ", xgb.score(X_train, y_train))

Training Accuracy:  1.0


In [ ]:
print("Testing Accuracy: ", accuracy_score(y_test, xgb.predict(X_test)))

Testing Accuracy:  0.9047619047619048


In [ ]:
result = pd.DataFrame(test_data['SKU'])
y_pred=xgb.predict(test_data)

In [ ]:
result["Customer_demand"] = y_pred
result["Actual Output:"]= y.tail(30)

result

,SKU,Customer_demand,Actual Output:
40,35,0,0
41,36,0,0
42,37,1,1
43,38,1,1
44,39,1,1
45,40,0,0
46,41,0,0
47,42,1,1
48,43,0,0
49,44,0,0


In [ ]:
joblib.dump(clf,'model.pkl')
#knn_from_pickle = pickle.loads(saved_model)
#knn_from_pickle.predict(X_test)


['model.pkl']